In [40]:
import polars as pl
import statsmodels.formula.api as smf
import numpy as np
from stargazer.stargazer import Stargazer

In [41]:
countries = set(open("./worldbankcountries.txt").read().split())

In [42]:
df = pl.read_parquet("../data/combined-2022-xgboost-synthetic-eiu.parquet")
df = df.filter(pl.col("country").str.contains_any(list(countries)))
pd_df = df.to_pandas()
prod_columns = {"math": "PISA Math", "in_math99": "PISA Math in global P99","in_math99:gdp_pc": "PISA Math 99 x GDP PC","imo_total_score": "IMO score per log population", "imo_total_score:gdp_pc": "IMO score x GDP pc", "arwu_ranked_num":"ARWU insitutions", "arwu_ranked_num:gdp_pc": "ARWU insitutions x GDP PC", "gdp_pc": "GDP per capita", "primary_completion": "Primary School Completion Rate", "lower_sec_completion": "Lower Sec. Completion Rate", "upper_sec_completion": "Upper Sec. Completion Rate", "democracy_eiu": "Democracy Rating", "population": "Population"}

In [43]:
pd_df["country"].isin(countries) == True

0       True
1       True
2       True
3       True
4       True
        ... 
4335    True
4336    True
4337    True
4338    True
4339    True
Name: country, Length: 4340, dtype: bool

In [44]:
pd_df.head()

,country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score,democracy_eiu,xg_primary_completion,xg_lower_sec_completion,xg_upper_sec_completion
0,AFG,2003,199.643228,92.702913,16.996843,15.810551,10.813185,22645130.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,3.310546,16.996843,15.810551,10.813185
1,AFG,2004,221.830531,-249.725464,15.907422,18.582813,10.807993,23553551.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,3.015871,15.907422,18.582813,10.807993
2,AFG,2005,254.115274,732.187384,24.646439,8.613730,6.516856,24411191.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,3.061568,24.646439,8.613730,6.516856
3,AFG,2006,274.015394,108.498820,27.276413,14.702160,10.832012,25442944.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,3.060000,27.276413,14.702160,10.832012
4,AFG,2007,376.318296,1180.338266,30.758774,9.329118,9.298284,25903301.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,2.632212,30.758774,9.329118,9.298284


In [45]:
df.shape

(4340, 19)

In [46]:
barrolee = pl.read_csv("../data/barrolee_imputed_nearest.csv")
bl_df = df.join(barrolee, on=["country", "year"], how="left")
pd_df = bl_df.to_pandas()
bl_df.shape

(4340, 23)

In [47]:
# pd_df["arwu_ranked_num"] = pd_df["arwu_ranked_num"] / pd_df["population"] * 1_000_000
# pd_df["imo_total_score"] = pd_df["imo_total_score"] * np.log(pd_df["population"])
# pd_df["arwu_ranked_num"] = np.where(pd_df["year"] < 2017, pd_df["arwu_ranked_num"], pd_df["arwu_ranked_num"] / 2)
# # pd_df["arwu_ranked_num"] = np.maximum(pd_df["arwu_ranked_num"], pd_df["arwu_sum_score"]) / pd_df["population"] * 1_000_000
# pd_df["gdp_pc_growth"] = 100 * pd_df["gdp_pc_growth"]
# pd_df["gdp_pc"] = pd_df["gdp_pc"] / 1000
# pd_df["population"] = np.log(pd_df["population"])
pd_df["high_income"] = np.where(pd_df["gdp_pc"] > 12475, 1, 0)

In [48]:
pd_df[["gdp_pc", "gdp_pc_growth", "math", "primary_completion", "lower_sec_completion", "population", "democracy_eiu"]].describe()

,gdp_pc,gdp_pc_growth,math,primary_completion,lower_sec_completion,population,democracy_eiu
count,4166.000000,4115.000000,441.000000,4340.000000,4340.000000,4.340000e+03,4340.000000
mean,16659.608725,194.424227,461.932227,73.194860,58.015136,3.300299e+07,6.426754
std,25384.750107,581.644421,56.230835,23.289931,24.986515,1.310632e+08,2.598427
min,114.367007,-5518.868118,315.963154,-1.092728,2.036170,9.668000e+03,0.320000
25%,1748.681408,-9.245650,416.137170,59.739037,39.721915,7.415448e+05,4.030000
50%,5851.470157,210.143326,478.696484,80.176689,58.758583,5.836896e+06,6.540000
75%,21672.068594,443.205546,502.800431,92.452991,79.256357,2.132160e+07,9.370000
max,240862.182448,9695.641990,574.663820,102.673096,104.400986,1.417173e+09,9.930000


In [49]:
pd_df[["in_math99", "arwu_ranked_num", "imo_total_score"]].describe()

,in_math99,arwu_ranked_num,imo_total_score
count,442.000000,4340.000000,4340.000000
mean,0.933362,0.078228,2.002553
std,1.457751,0.217070,3.096665
min,0.000000,0.000000,0.000000
25%,0.056695,0.000000,0.000000
50%,0.486001,0.000000,0.000000
75%,1.184127,0.000000,3.823251
max,14.639147,1.565903,14.306904


In [50]:
# pd_df.groupby("year")["arwu_ranked_num"].sum()

In [51]:
# math = smf.ols("math ~ gdp_pc + imo_total_score + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu", pd_df[pd_df["year"].isin([2003, 2006, 2009, 2012, 2015, 2018])]).fit()
# math.summary()

In [52]:
pisa_df = pd_df[pd_df["year"].isin([2003, 2006, 2009, 2012, 2015, 2018, 2022]) & (pd_df["math"].notnull())]

In [53]:
pisa_df.describe()

,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,...,imo_total_score,democracy_eiu,xg_primary_completion,xg_lower_sec_completion,xg_upper_sec_completion,yr_sch,yr_sch_pri,yr_sch_sec,yr_sch_ter,high_income
count,441.000000,441.000000,440.000000,441.000000,441.000000,441.000000,4.410000e+02,441.000000,441.000000,441.000000,...,441.000000,441.000000,441.000000,441.000000,441.000000,407.000000,407.000000,407.000000,407.000000,441.000000
mean,2013.111111,28463.985814,169.064165,90.000496,77.894528,62.200361,3.531860e+07,0.256697,57.672109,461.932227,...,4.724749,7.246516,89.797760,77.984741,62.319782,10.682840,5.712914,4.192098,0.777830,0.632653
std,6.057594,25255.097112,452.675266,10.245705,17.143303,18.482788,5.944562e+07,0.334478,266.675308,56.230835,...,3.486964,1.772025,10.144611,16.071438,17.290270,1.693632,0.927824,1.173185,0.375497,0.482630
min,2003.000000,543.110702,-2292.684929,51.353222,29.213850,18.444260,3.400000e+04,0.000000,0.000000,315.963154,...,0.000000,1.930000,47.056778,34.772507,20.729521,4.873000,3.359000,1.049000,0.061000,0.000000
25%,2009.000000,8495.653081,11.029820,82.985069,63.849010,47.574020,4.408581e+06,0.000000,0.000000,416.137170,...,1.540440,6.412529,82.849083,65.300110,48.126324,9.657000,5.272500,3.339000,0.499000,0.000000
50%,2012.000000,19861.697430,190.810046,94.491341,83.070015,65.789413,9.799186e+06,0.078525,0.000000,478.696484,...,4.884144,7.570000,94.278702,83.165688,64.813316,11.051000,5.724000,4.220000,0.721000,1.000000
75%,2018.000000,42801.908117,396.299473,98.383331,91.896492,77.485718,4.218764e+07,0.460051,25.700000,502.800431,...,7.505591,8.380000,97.650375,91.148918,76.888718,11.975500,5.969500,5.090000,0.995000,1.000000
max,2022.000000,149461.785571,3303.048777,101.947922,101.966637,97.399788,3.332876e+08,1.536120,2446.000000,574.663820,...,11.786918,9.930000,101.947922,102.010117,97.183167,13.275000,8.866000,7.915000,1.846000,1.000000


In [54]:
pisa_df[["in_math99", "math", "arwu_ranked_num", "imo_total_score"]].corr()

,in_math99,math,arwu_ranked_num,imo_total_score
in_math99,1.000000,0.641199,0.364180,0.254865
math,0.641199,1.000000,0.559003,0.202928
arwu_ranked_num,0.364180,0.559003,1.000000,-0.013410
imo_total_score,0.254865,0.202928,-0.013410,1.000000


In [55]:
pisa_no_controls = smf.ols("gdp_pc_growth ~ imo_total_score + arwu_ranked_num + in_math99", pisa_df).fit()
pisa_no_fix = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu", pisa_df).fit()
pisa_no_vars = smf.ols("gdp_pc_growth ~ gdp_pc + math + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu", pisa_df).fit()
pisa = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + C(year) ", pisa_df).fit()
pisa_country = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + country + C(year)", pisa_df).fit()

In [56]:
pisa.f_test("imo_total_score = 0, arwu_ranked_num = 0, in_math99 = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=3.5771223192762687, p=0.014033043990831988, df_denom=423, df_num=3>

In [57]:
non_pisa_pyears = smf.ols("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + C(year) ", pd_df[pd_df["year"].isin([2003, 2006, 2009, 2012, 2015, 2018, 2022])]).fit()
non_pisa = smf.ols("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + C(year) ", pd_df).fit()
non_pisa_country = smf.ols("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + C(year) + country", pd_df).fit()
# non_pisa_no_interac = smf.ols("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + C(year) ", pd_df).fit()

In [68]:
pisa_sg = Stargazer([pisa_no_controls, pisa_no_fix, pisa, pisa_country])
pisa_sg.covariate_order(["in_math99", "imo_total_score", "arwu_ranked_num", "gdp_pc", "math", "primary_completion", "lower_sec_completion", "upper_sec_completion", "democracy_eiu", "population"])
pisa_sg.add_line('Time Effects', ['No', 'No', 'Yes', 'Yes'])
pisa_sg.add_line('Fixed Effects', ['No', 'No', 'No', 'Yes'])
pisa_sg.add_line('Controls', ['No', 'Yes', 'Yes', 'Yes'])
pisa_sg.add_line('Entities', [89, 89, 89, 89])
pisa_sg.rename_covariates(prod_columns)
pisa_sg.custom_columns(['Model 1 (base)', 'Model 2', 'Model 3 (Time FE)', 'Model 4 (Time + Entity FE)'], [1, 1, 1, 1])
pisa_sg.show_model_numbers(False)
print(pisa_sg.render_latex())
# pisa_sg

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{4}{c}{\textit{Dependent variable: gdp_pc_growth}} \
\cr \cline{2-5}
\\[-1.8ex] & \multicolumn{1}{c}{Model 1 (base)} & \multicolumn{1}{c}{Model 2} & \multicolumn{1}{c}{Model 3 (Time FE)} & \multicolumn{1}{c}{Model 4 (Time + Entity FE)}  \\
\hline \\[-1.8ex]
 PISA Math in global P99 & -31.228$^{*}$ & -38.928$^{*}$ & -34.569$^{*}$ & -115.492$^{***}$ \\
& (16.314) & (20.802) & (17.850) & (36.599) \\
 IMO score per log population & 10.674$^{*}$ & 4.924$^{}$ & 3.122$^{}$ & -11.158$^{}$ \\
& (6.363) & (7.470) & (6.441) & (15.728) \\
 ARWU insitutions & -166.641$^{**}$ & -162.705$^{*}$ & -176.049$^{**}$ & -159.960$^{}$ \\
& (68.800) & (86.871) & (72.281) & (112.884) \\
 GDP per capita & & -0.001$^{}$ & -0.001$^{}$ & 0.003$^{}$ \\
& & (0.001) & (0.001) & (0.003) \\
 PISA Math & & 0.619$^{}$ & 0.425$^{}$ & 0.178$^{}$ \\
& & (0.716) & (0.684) & (1.735) \\
 Primary School Co

In [65]:
pisa_no_math = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + C(year) ", pisa_df).fit()
non_pisa_sg = Stargazer([pisa, pisa_no_math, non_pisa_pyears, non_pisa, non_pisa_country])
non_pisa_sg.covariate_order(["imo_total_score", "arwu_ranked_num", "gdp_pc", "math", "primary_completion", "lower_sec_completion", "upper_sec_completion", "democracy_eiu", "population"])
non_pisa_sg.add_line('Time Effects', ['Yes', 'Yes', 'Yes', 'Yes', 'Yes'])
non_pisa_sg.add_line('Fixed Effects', ['No', 'No', 'No', 'No', 'Yes'])
non_pisa_sg.add_line('Entities', [49, 49, 213, 213, 213])
non_pisa_sg.custom_columns(['Model 3 (PISA)', 'PISA countries exc. math', 'PISA years', 'All years', 'All years, FE'], [1, 1, 1, 1, 1])
non_pisa_sg.show_model_numbers(False)
non_pisa_sg.rename_covariates(prod_columns)
print(non_pisa_sg.render_latex())
# non_pisa_sg

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable: gdp_pc_growth}} \
\cr \cline{2-6}
\\[-1.8ex] & \multicolumn{1}{c}{Model 3 (PISA)} & \multicolumn{1}{c}{PISA countries exc. math} & \multicolumn{1}{c}{PISA years} & \multicolumn{1}{c}{All years} & \multicolumn{1}{c}{All years, FE}  \\
\hline \\[-1.8ex]
 IMO score per log population & 3.122$^{}$ & -0.460$^{}$ & 3.647$^{}$ & 17.239$^{***}$ & -9.490$^{}$ \\
& (6.441) & (5.913) & (6.516) & (3.546) & (8.562) \\
 ARWU insitutions & -176.049$^{**}$ & -196.841$^{***}$ & -183.403$^{**}$ & -191.701$^{***}$ & -342.927$^{***}$ \\
& (72.281) & (71.171) & (82.042) & (44.185) & (87.248) \\
 GDP per capita & -0.001$^{}$ & -0.002$^{**}$ & -0.001$^{*}$ & -0.001$^{*}$ & 0.008$^{***}$ \\
& (0.001) & (0.001) & (0.001) & (0.000) & (0.002) \\
 PISA Math & 0.425$^{}$ & & & & \\
& (0.684) & & & & \\
 Primary School Completion Rate & -6.467$^{*}$ & -5.609

In [21]:
group1 = pd_df[(pd_df["arwu_ranked_num"].notnull()) & (pd_df["math"].notnull()) & (pd_df["imo_total_score"].notnull()) & (pd_df["primary_completion"].notnull()) & (pd_df["lower_sec_completion"].notnull()) & (pd_df["upper_sec_completion"].notnull()) & (pd_df["democracy_eiu"].notnull())]
group1.shape

(441, 24)

In [22]:
group2 = pd_df[(pd_df["gdp_pc"].notnull()) & (pd_df["arwu_ranked_num"].notnull()) & (pd_df["imo_total_score"].notnull()) & (pd_df["primary_completion"].notnull()) & (pd_df["lower_sec_completion"].notnull()) & (pd_df["upper_sec_completion"].notnull()) & (pd_df["democracy_eiu"].notnull())]
group2.shape

(4166, 24)

In [23]:
group2[["gdp_pc", "gdp_pc_growth", "math", "primary_completion", "lower_sec_completion", "population", "democracy_eiu"]].describe()

,gdp_pc,gdp_pc_growth,math,primary_completion,lower_sec_completion,population,democracy_eiu
count,4166.000000,4094.000000,441.000000,4166.000000,4166.000000,4.166000e+03,4166.000000
mean,16659.608725,194.928618,461.932227,73.091580,58.501928,3.411634e+07,6.377357
std,25384.750107,573.064019,56.230835,23.590501,25.191979,1.336416e+08,2.548797
min,114.367007,-5518.868118,315.963154,-1.092728,2.036170,9.668000e+03,0.320000
25%,1748.681408,-7.236680,416.137170,58.920576,39.755115,9.061522e+05,4.030000
50%,5851.470157,211.621635,478.696484,80.162407,59.955118,6.085110e+06,6.482334
75%,21672.068594,443.845893,502.800431,92.660107,79.804070,2.168614e+07,9.080000
max,240862.182448,9695.641990,574.663820,102.673096,104.400986,1.417173e+09,9.930000


In [24]:
group2[["arwu_ranked_num", "imo_total_score"]].describe()

,arwu_ranked_num,imo_total_score
count,4166.000000,4166.000000
mean,0.081473,2.052976
std,0.220959,3.113608
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,3.973770
max,1.565903,14.306904


In [25]:
group1[["gdp_pc", "gdp_pc_growth", "math", "primary_completion", "lower_sec_completion", "population", "democracy_eiu"]].describe()

,gdp_pc,gdp_pc_growth,math,primary_completion,lower_sec_completion,population,democracy_eiu
count,441.000000,440.000000,441.000000,441.000000,441.000000,4.410000e+02,441.000000
mean,28463.985814,169.064165,461.932227,90.000496,77.894528,3.531860e+07,7.246516
std,25255.097112,452.675266,56.230835,10.245705,17.143303,5.944562e+07,1.772025
min,543.110702,-2292.684929,315.963154,51.353222,29.213850,3.400000e+04,1.930000
25%,8495.653081,11.029820,416.137170,82.985069,63.849010,4.408581e+06,6.412529
50%,19861.697430,190.810046,478.696484,94.491341,83.070015,9.799186e+06,7.570000
75%,42801.908117,396.299473,502.800431,98.383331,91.896492,4.218764e+07,8.380000
max,149461.785571,3303.048777,574.663820,101.947922,101.966637,3.332876e+08,9.930000


In [26]:
group1[["in_math99", "arwu_ranked_num", "imo_total_score"]].describe()

,in_math99,arwu_ranked_num,imo_total_score
count,441.000000,441.000000,441.000000
mean,0.935478,0.256697,4.724749
std,1.458726,0.334478,3.486964
min,0.000000,0.000000,0.000000
25%,0.061045,0.000000,1.540440
50%,0.486487,0.078525,4.884144
75%,1.185308,0.460051,7.505591
max,14.639147,1.536120,11.786918


In [66]:
year_pisa_regs = []
average = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu", pisa_df.groupby("country").mean().reset_index()).fit()
years = list(range(2003, 2019, 3)) + [2022]
years
for y in years:
    filtered = pisa_df[pisa_df["year"] == y]
    reg = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu", filtered).fit()
    year_pisa_regs.append(reg)
year_pisa_regs.append(pisa_country)

In [67]:
yearly_pisa = Stargazer(year_pisa_regs)
yearly_pisa.covariate_order(["in_math99", "imo_total_score", "arwu_ranked_num", "gdp_pc", "math", "primary_completion", "lower_sec_completion", "upper_sec_completion", "democracy_eiu", "population"])
yearly_pisa.rename_covariates(prod_columns)
yearly_pisa.custom_columns(list(map(str, years)) + ["Panel FE"], [1] * len(year_pisa_regs))
yearly_pisa.show_model_numbers(False)
# print(pisa_sg.render_latex())
yearly_pisa.add_line('Controls', ['Yes'] * len(year_pisa_regs))
print(yearly_pisa.render_latex())
# yearly_pisa

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{8}{c}{\textit{Dependent variable: gdp_pc_growth}} \
\cr \cline{2-9}
\\[-1.8ex] & \multicolumn{1}{c}{2003} & \multicolumn{1}{c}{2006} & \multicolumn{1}{c}{2009} & \multicolumn{1}{c}{2012} & \multicolumn{1}{c}{2015} & \multicolumn{1}{c}{2018} & \multicolumn{1}{c}{2022} & \multicolumn{1}{c}{Panel FE}  \\
\hline \\[-1.8ex]
 PISA Math in global P99 & -52.490$^{}$ & -193.475$^{**}$ & 154.062$^{**}$ & 16.575$^{}$ & -34.309$^{}$ & 3.769$^{}$ & -69.681$^{**}$ & -115.492$^{***}$ \\
& (48.524) & (84.559) & (62.680) & (36.365) & (76.140) & (27.106) & (27.702) & (36.599) \\
 IMO score per log population & 4.478$^{}$ & -9.112$^{}$ & 14.038$^{}$ & 7.368$^{}$ & -16.622$^{}$ & 2.193$^{}$ & -0.174$^{}$ & -11.158$^{}$ \\
& (17.327) & (20.741) & (17.012) & (13.983) & (27.567) & (7.722) & (13.474) & (15.728) \\
 ARWU insitutions & -300.038$^{**}$ & -454.401$^{***}$ & 136.623$^{}$

# Without synthetic data

In [34]:
no_impute_df = pl.read_parquet("../data/combined-2022-included.parquet")
ni_pd_df = no_impute_df.to_pandas()
prod_columns = {"math": "PISA Math", "in_math99": "PISA Math in global P99","in_math99:gdp_pc": "PISA Math 99 x GDP PC","imo_total_score": "IMO score per log population", "imo_total_score:gdp_pc": "IMO score x GDP pc", "arwu_ranked_num":"ARWU insitutions", "arwu_ranked_num:gdp_pc": "ARWU insitutions x GDP PC", "gdp_pc": "GDP per capita", "primary_completion": "Primary School Completion Rate", "lower_sec_completion": "Lower Sec. Completion Rate", "upper_sec_completion": "Upper Sec. Completion Rate", "democracy_eiu": "Democracy Rating", "population": "Population"}
ni_pd_df["arwu_ranked_num"] = ni_pd_df["arwu_ranked_num"] / ni_pd_df["population"] * 1_000_000
ni_pd_df["imo_total_score"] = ni_pd_df["imo_total_score"] / np.log(ni_pd_df["population"])
ni_pd_df["arwu_ranked_num"] = np.where(ni_pd_df["year"] < 2017, ni_pd_df["arwu_ranked_num"], ni_pd_df["arwu_ranked_num"] / 2)
# ni_pd_df["arwu_ranked_num"] = np.maximum(ni_pd_df["arwu_ranked_num"], ni_pd_df["arwu_sum_score"]) / ni_pd_df["population"] * 1_000_000
ni_pd_df["gdp_pc_growth"] = 100 * ni_pd_df["gdp_pc_growth"]
# pd_df["gdp_pc"] = pd_df["gdp_pc"] / 1000
# pd_df["population"] = np.log(pd_df["population"])

In [35]:
ni_pisa_df = ni_pd_df[ni_pd_df["year"].isin([2003, 2006, 2009, 2012, 2015, 2018, 2022]) & (ni_pd_df["math"].notnull())].copy()

In [36]:
ni_pisa_df["no_impute"] = np.where(ni_pisa_df["primary_completion"].notnull() & ni_pisa_df["lower_sec_completion"].notnull() & ni_pisa_df["upper_sec_completion"].notnull() & ni_pisa_df["democracy_eiu"].notnull(), 1, 0)


In [37]:
pisa_no_controls = smf.ols("gdp_pc_growth ~ imo_total_score + arwu_ranked_num + in_math99", ni_pisa_df).fit()
pisa_no_fix = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu", ni_pisa_df).fit()
pisa = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + C(year) ", ni_pisa_df).fit()
pisa_country = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + country + C(year)", ni_pisa_df).fit()
pisa_sg = Stargazer([pisa_no_controls, pisa_no_fix, pisa, pisa_country])
pisa_sg.covariate_order(["in_math99", "imo_total_score", "arwu_ranked_num"])
pisa_sg.add_line('Time Effects', ['No', 'No', 'Yes', 'Yes'])
pisa_sg.add_line('Fixed Effects', ['No', 'No', 'No', 'Yes'])
pisa_sg.add_line('Controls', ['No', 'Yes', 'Yes', 'Yes'])
pisa_sg.rename_covariates(prod_columns)
pisa_sg.custom_columns(['Model 1 (base)', 'Model 2', 'Model 3 (Time FE)', 'Model 4 (Time + Entity FE)'], [1, 1, 1, 1])
pisa_sg.show_model_numbers(False)
print(pisa_sg.render_latex())
# pisa_sg

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{4}{c}{\textit{Dependent variable: gdp_pc_growth}} \
\cr \cline{2-5}
\\[-1.8ex] & \multicolumn{1}{c}{Model 1 (base)} & \multicolumn{1}{c}{Model 2} & \multicolumn{1}{c}{Model 3 (Time FE)} & \multicolumn{1}{c}{Model 4 (Time + Entity FE)}  \\
\hline \\[-1.8ex]
 PISA Math in global P99 & -31.228$^{*}$ & 5.985$^{}$ & 8.659$^{}$ & 25.421$^{}$ \\
& (16.314) & (22.468) & (21.493) & (84.291) \\
 IMO score per log population & 10.674$^{*}$ & 6.824$^{}$ & 12.529$^{}$ & 50.506$^{}$ \\
& (6.363) & (9.440) & (8.631) & (32.187) \\
 ARWU insitutions & -166.641$^{**}$ & -190.750$^{}$ & -249.208$^{**}$ & -454.384$^{}$ \\
& (68.800) & (127.229) & (116.476) & (351.559) \\
 Time Effects & No & No & Yes & Yes \\
 Fixed Effects & No & No & No & Yes \\
 Controls & No & Yes & Yes & Yes \\
\hline \\[-1.8ex]
 Observations & 440 & 112 & 112 & 112 \\
 $R^2$ & 0.037 & 0.204 & 0.383 & 0.715 \\


In [38]:
filtered = pisa_df.copy()
filtered["no_impute"] = ni_pisa_df["no_impute"]
filtered = filtered[filtered["no_impute"] == 1]
pisa_no_controls = smf.ols("gdp_pc_growth ~ imo_total_score + arwu_ranked_num + in_math99", pisa_df).fit()
pisa_no_fix = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + xg_primary_completion + xg_lower_sec_completion + xg_upper_sec_completion + population + democracy_eiu", filtered).fit()
pisa = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + xg_primary_completion + xg_lower_sec_completion + xg_upper_sec_completion + population + democracy_eiu + C(year) ", filtered).fit()
pisa_country = smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num + xg_primary_completion + xg_lower_sec_completion + xg_upper_sec_completion + population + democracy_eiu + country + C(year)", filtered).fit()
pisa_sg = Stargazer([pisa_no_controls, pisa_no_fix, pisa, pisa_country])
pisa_sg.covariate_order(["in_math99", "imo_total_score", "arwu_ranked_num"])
pisa_sg.add_line('Time Effects', ['No', 'No', 'Yes', 'Yes'])
pisa_sg.add_line('Fixed Effects', ['No', 'No', 'No', 'Yes'])
pisa_sg.add_line('Controls', ['No', 'Yes', 'Yes', 'Yes'])
pisa_sg.rename_covariates(prod_columns)
pisa_sg.custom_columns(['Model 1 (base)', 'Model 2', 'Model 3 (Time FE)', 'Model 4 (Time + Entity FE)'], [1, 1, 1, 1])
pisa_sg.show_model_numbers(False)
print(pisa_sg.render_latex())
# pisa_sg

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{4}{c}{\textit{Dependent variable: gdp_pc_growth}} \
\cr \cline{2-5}
\\[-1.8ex] & \multicolumn{1}{c}{Model 1 (base)} & \multicolumn{1}{c}{Model 2} & \multicolumn{1}{c}{Model 3 (Time FE)} & \multicolumn{1}{c}{Model 4 (Time + Entity FE)}  \\
\hline \\[-1.8ex]
 PISA Math in global P99 & -31.228$^{*}$ & -2.504$^{}$ & 0.536$^{}$ & -39.980$^{}$ \\
& (16.314) & (22.218) & (21.288) & (70.960) \\
 IMO score per log population & 10.674$^{*}$ & 9.569$^{}$ & 12.465$^{}$ & 39.725$^{}$ \\
& (6.363) & (9.055) & (8.258) & (37.400) \\
 ARWU insitutions & -166.641$^{**}$ & -258.397$^{**}$ & -306.807$^{***}$ & -336.747$^{}$ \\
& (68.800) & (124.418) & (114.960) & (384.764) \\
 Time Effects & No & No & Yes & Yes \\
 Fixed Effects & No & No & No & Yes \\
 Controls & No & Yes & Yes & Yes \\
\hline \\[-1.8ex]
 Observations & 440 & 112 & 112 & 112 \\
 $R^2$ & 0.037 & 0.262 & 0.418 & 0.70

In [132]:
ni_pisa_df[(ni_pisa_df["primary_completion"].notnull()) & (ni_pisa_df["democracy_eiu"].notnull()) & (ni_pd_df["lower_sec_completion"].notnull()) & (ni_pisa_df["upper_sec_completion"].notnull())][["gdp_pc", "math", "arwu_ranked_num", "imo_total_score", "in_math99", "math"]].describe()

/var/folders/yy/l2lt4q6j40s7ty3hj7122pdw0000gn/T/ipykernel_14950/485440488.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,gdp_pc,math,arwu_ranked_num,imo_total_score,in_math99,math
count,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,25075.596453,456.916979,0.220021,4.640821,0.912355,456.916979
std,20846.730295,55.105992,0.292263,3.192320,1.662720,55.105992
min,1572.797940,327.250491,0.000000,0.000000,0.000000,327.250491
25%,8965.398345,410.003171,0.000000,1.736566,0.027980,410.003171
50%,17664.026111,473.008940,0.041274,5.139991,0.368485,473.008940
75%,39246.543188,497.459671,0.365121,6.845541,1.046649,497.459671
max,98041.362238,568.359669,1.211386,11.257523,9.501632,568.359669


In [133]:
pisa_df[["gdp_pc", "math", "arwu_ranked_num", "imo_total_score", "in_math99", "math"]].describe()

,gdp_pc,math,arwu_ranked_num,imo_total_score,in_math99,math
count,441.000000,441.000000,441.000000,441.000000,441.000000,441.000000
mean,28463.985814,461.932227,0.256697,4.724749,0.935478,461.932227
std,25255.097112,56.230835,0.334478,3.486964,1.458726,56.230835
min,543.110702,315.963154,0.000000,0.000000,0.000000,315.963154
25%,8495.653081,416.137170,0.000000,1.540440,0.061045,416.137170
50%,19861.697430,478.696484,0.078525,4.884144,0.486487,478.696484
75%,42801.908117,502.800431,0.460051,7.505591,1.185308,502.800431
max,149461.785571,574.663820,1.536120,11.786918,14.639147,574.663820


In [32]:
ni_pisa_df[(ni_pisa_df["primary_completion"].notnull()) & (ni_pisa_df["democracy_eiu"].notnull()) & (ni_pd_df["lower_sec_completion"].notnull()) & (ni_pisa_df["upper_sec_completion"].notnull())]["country"].value_counts().reset_index()

/var/folders/yy/l2lt4q6j40s7ty3hj7122pdw0000gn/T/ipykernel_42509/3651639314.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,country,count
0,DEU,4
1,URY,4
2,BRA,4
3,COL,4
4,PRT,4
5,USA,4
6,MEX,4
7,ESP,4
8,HKG,3
9,TUR,3
